In [1]:
from astropy import units as u
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy.io import ascii
from astropy.table import setdiff, Table, unique, vstack
import numpy as np
%run Crossmatch.ipynb
%run Utility.ipynb

In [2]:
## NEW CELL WITH SEQUENTIAL AVERAGING ##
%run Crossmatch.ipynb
total_duplicates = Table()

# Path for aperture photometry based PSF-modelled catalogs
path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_Aper_Model_Individually/'

averaged = ascii.read('/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_Aper_Model_Individually/Chihiro.dat')

# Encodes the path of the dataset to be used by other OS methods

folder = ['Clarisse', 'Fio', 'Kiki', 'Nausicaa', 'Ponyo', 'San', 'Satsuki', 'Sheeta', 'Sophie']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    item = path + item + '.dat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['I_MAG_SUBARU'] = (temp_duplicates['I_MAG_SUBARU'] + temp_duplicates['I_MAG_SUBARU_2']) / 2
        
        temp_duplicates['I_MAG_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_MAG_SUBARU_ERR'] * temp_duplicates['I_MAG_SUBARU_ERR'] + 0.25 * temp_duplicates['I_MAG_SUBARU_ERR_2'] * temp_duplicates['I_MAG_SUBARU_ERR_2'])
        
        temp_duplicates['I_FLUX_SUBARU'] = (temp_duplicates['I_FLUX_SUBARU'] + temp_duplicates['I_FLUX_SUBARU_2']) / 2
        
        temp_duplicates['I_FLUX_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_FLUX_SUBARU_ERR'] * temp_duplicates['I_FLUX_SUBARU_ERR'] + 0.25 * temp_duplicates['I_FLUX_SUBARU_ERR_2'] * temp_duplicates['I_FLUX_SUBARU_ERR_2'])
        
        temp_duplicates['I_FLUX_RADIUS_SUBARU'] = (temp_duplicates['I_FLUX_RADIUS_SUBARU'] + temp_duplicates['I_FLUX_RADIUS_SUBARU_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['I_SNR_WIN_SUBARU'] = (temp_duplicates['I_SNR_WIN_SUBARU'] + temp_duplicates['I_SNR_WIN_SUBARU_2']) / 2
        
        #temp_duplicates['FLAGS'] = temp_duplicates['FLAGS'] + temp_duplicates['FLAGS_2']
        
        #temp_duplicates['FLAGS_MODEL'] = temp_duplicates['FLAGS_MODEL'] + temp_duplicates['FLAGS_MODEL_2']
        
        #temp_duplicates['I_FLUX_APER_SUBARU'] = (temp_duplicates['I_FLUX_APER_SUBARU'] + temp_duplicates['I_FLUX_APER_SUBARU_2']) / 2
        
        #temp_duplicates['I_FLUX_APER_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_FLUX_APER_SUBARU_ERR'] * temp_duplicates['I_FLUX_APER_SUBARU_ERR'] + 0.25 * temp_duplicates['I_FLUX_APER_SUBARU_ERR_2'] * temp_duplicates['I_FLUX_APER_SUBARU_ERR_2'])
        
        # Removes the columns from the cross-matched second catalog
        
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])


# Path for aperture photometry based PSF-modelled catalogs
ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/Full_Catalog_ZP_CT_Aper_PSF.dat', format = 'csv', overwrite = True)



In [3]:
## The duplicates code for the H-Alpha catalog using last cell's method

%run Crossmatch.ipynb
total_duplicates = Table()

# The path where the dataset is located

path = '/home/yobd/Documents/AstroAliSciObj/N-A-L656/MINAREA_4_THRESH_2_dot_7_SNR_20_SATUR_LEVEL_B4_PSFEX_30000_SMP_VAR_0_dot_2_SEEING_6_dot_7_7_MAG_PSF/Catalogs_Finished/'

averaged = ascii.read('/home/yobd/Documents/AstroAliSciObj/N-A-L656/MINAREA_4_THRESH_2_dot_7_SNR_20_SATUR_LEVEL_B4_PSFEX_30000_SMP_VAR_0_dot_2_SEEING_6_dot_7_7_MAG_PSF/Catalogs_Finished/N-A-L656_chihiro_stacked_tmp.dat')

# Encodes the path of the dataset to be used by other OS methods

folder = ['N-A-L656_clarisse_stacked_tmp', 'N-A-L656_fio_stacked_tmp', 'N-A-L656_kiki_stacked_tmp', 'N-A-L656_nausicaa_stacked_tmp', 'N-A-L656_ponyo_stacked_tmp', 'N-A-L656_san_stacked_tmp', 'N-A-L656_satsuki_stacked_tmp', 'N-A-L656_sheeta_stacked_tmp', 'N-A-L656_sophie_stacked_tmp']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    item = path + item + '.dat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['FLUX_PSF'] = (temp_duplicates['FLUX_PSF'] + temp_duplicates['FLUX_PSF_2']) / 2
        
        temp_duplicates['FLUXERR_PSF'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_PSF'] * temp_duplicates['FLUXERR_PSF'] + 0.25 * temp_duplicates['FLUXERR_PSF_2'] * temp_duplicates['FLUXERR_PSF_2'])
        
        temp_duplicates['FLUX_RADIUS'] = (temp_duplicates['FLUX_RADIUS'] + temp_duplicates['FLUX_RADIUS_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['SNR_WIN'] = (temp_duplicates['SNR_WIN'] + temp_duplicates['SNR_WIN_2']) / 2
        
        temp_duplicates['FLAGS'] = temp_duplicates['FLAGS'] + temp_duplicates['FLAGS_2']
        
        temp_duplicates['FLAGS_MODEL'] = temp_duplicates['FLAGS_MODEL'] + temp_duplicates['FLAGS_MODEL_2']
        
        # Removes the columns from the cross-matched second catalog
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])

averaged.remove_rows([averaged['FLAGS'] != 0])

ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/N-A-L656_Full_Catalog_Aper_PSF.dat', format = 'csv', overwrite = True)



/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False
